In [6]:
from numpy.random import default_rng

In [11]:
rng = default_rng()
numbers = rng.choice(20, size=10, replace=False)

In [9]:
numbers

array([ 0,  7,  1, 17, 11, 13,  4,  2, 16,  8])

In [12]:
import torch

In [53]:
total_size = 130203
rank = 0
world_size = 4

indices = list(range(total_size))

In [55]:
indices_0 = indices[0:total_size:world_size]
indices_1 = indices[1:total_size:world_size]
indices_2 = indices[2:total_size:world_size]
indices_3 = indices[3:total_size:world_size]

In [56]:
len(indices_0), len(indices_1), len(indices_2), len(indices_3) 

(32551, 32551, 32551, 32550)

In [1]:
"""
4차 3번 : 재활용
"""

import json
import sys
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix

# 평가 : Macro F1-Score

# Our f1 score (class 별로 f1score x)

In [14]:
def our_f1score():
    with open('./dataset/trash/val/valcoco.json') as f:
        data_dict = json.load(f)
    realdata = data_dict['annotations']

    real = {}
    for data in realdata:
        try:
            if real[data['image_id']]:
                if data['category_id'] in real[data['image_id']] :                
                    continue
                else :
                    real[data['image_id']].append(data['category_id'])
        except:
            real[data['image_id']] = [data['category_id']]

    # predict
    with open('./result.bbox.json') as f:
        pred = json.load(f)
        
    predict = {}
    for data in pred:
        try:
            if predict[data['image_id']]: 
                if data['category_id'] in predict[data['image_id']] :                
                    continue
                else :
                    predict[data['image_id']].append(data['category_id'])
        except:
            predict[data['image_id']] = [data['category_id']]
    TP = 0
    FP = 0
    FN = 0
    for index in real.keys(): # image index
        try:
            p = set(predict[index])
        except:
            p = set()

        r = set(real[index])
        
        TP += len(r&p)
        FP += (len(p) - len(r&p))
        FN += (len(r) - len(r&p))

    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1score = 2*(precision * recall)/ (precision + recall)
    print('f1 score = %.4f | TP = %d, FP = %d, FN = %d' %(f1score, TP, FP, FN))

# AI grand challenge f1 score(class별 f1 score)

In [2]:
def string_to_list(input_string):
    # for i in input_string:
    #     print(i)
    tmp = ""
    for i in input_string:
        tmp += i
    string_dict_list = tmp.strip("[]")
    # print(string_dict_list)

    data = [x.strip("}") for x in string_dict_list.split(',')]
    data = [x.split(":")[-1].strip() for x in data]
    
    return True

In [3]:
def get_annotation(total_label, input_dict):
    sorted_list = []
    for i in input_dict["annotations"]:
        one_hot = [0 for x in range(len(total_label))]
        file_name = i["file_name"]
        labels = []
        if len(i["object"]) > 0:
            for obj in i["object"]:
                if len(obj["box"]) > 0:
                    labels.append(obj["label"].replace("_", ""))

        unq_labels = sorted(list(set(labels)))
        if len(unq_labels) > 0:
            for unq in unq_labels:
                one_hot[total_label.index(unq)] = 1

        sorted_list.append([file_name, one_hot])

    sorted_list = sorted(sorted_list, key=lambda k:k[0])

    return np.array([x[1] for x in sorted_list])

In [4]:
def to_code(onehot):
    codes = []
    for i in onehot:
        code = ""
        for j in i:
            code += str(j)
        codes.append(code)

    return codes

In [5]:
def track3_evaluation(pred, gt):
    # Json 형식
    with open(pred) as f: # 예측 파일
        pred_dict = json.load(f)
    with open(gt) as f: # 정답 파일
        gt_dict = json.load(f)

    # convert coco to 2020AGC
    gt_dict = val_convert(gt_dict)
        
    # Get gournd-truth label list
    total_labels = []
    for gt in gt_dict["annotations"]:
        for obj in gt["object"]:
            total_labels.append(obj["label"].replace("_", ""))
    unq_label = sorted(list(set(total_labels)))

    # one hot 인코딩된 배열
    y_pred = get_annotation(unq_label, pred_dict)
    y_true = get_annotation(unq_label, gt_dict)
    
    multi_conf = multilabel_confusion_matrix(y_true, y_pred)
    
    """
    # 이미지 단위 f1-score
    pred_code = to_code(y_pred)
    gt_code = to_code(y_true)

    
    """
    f1 = f1_score(y_true, y_pred, average='macro')
    print("score:{}".format(f1))
    
    return multi_conf, y_pred, y_true

    # f1 = f1_score(y_true, y_pred, average='micro') 

In [6]:
def val_convert(gt_dict):
    gt_list = [{} for _ in range(452)]

    for idx, img in enumerate(gt_dict['images']):
        gt_list[idx]['id'] = img['id']
        gt_list[idx]['file_name'] = img['file_name']
        gt_list[idx]['object'] = []
        for obj in gt_dict['annotations']:
            if img['id'] == obj['image_id']:
                obj_sample = {}
                obj_sample['box'] = obj['segmentation'][0]
                obj_sample['label'] = 'c'+str(obj['category_id'])
                gt_list[idx]['object'].append(obj_sample)
    result = {'annotations':gt_list}
    
    return result

In [7]:
# get iou from coco (x,y,w,h)
def get_iou(pred_list, real_list):
    pts_p = [pred_list[0] - pred_list[2]/2, pred_list[1] - pred_list[3]/2, 
             pred_list[0] + pred_list[2]/2, pred_list[1] + pred_list[3]/2]
    
    pts_r = [real_list[0] - real_list[2]/2, real_list[1] - real_list[3]/2,
             real_list[0] + real_list[2]/2, real_list[1] + real_list[3]/2]
    
    x_left = max(pts_p[0], pts_r[0])
    y_top = max(pts_p[1], pts_r[1])
    x_right = min(pts_p[2], pts_r[2])
    y_bottom = min(pts_p[3], pts_r[3])
    
    intersection = (x_right - x_left) * (y_bottom - y_top)
    
    pred_area = (pts_p[2] - pts_p[0]) * (pts_p[3] - pts_p[1])
    real_area = (pts_r[2] - pts_r[0]) * (pts_r[3] - pts_r[1])
    
    iou = intersection/(pred_area+real_area-intersection)
    
    return iou

In [15]:
multi_conf, y_pred, y_true = track3_evaluation('t3_res_0030.json', 'dataset/trash/val/valcoco.json')

score:0.763309663362174


In [51]:
b = y_true-y_pred
b.sum(axis=0)

array([ 11,  16,  11,   2,   9, -10, -80])

In [52]:
a = 0
for i in b:
    if i.sum() != 0:
        print(i)
        a +=i

[0 0 1 0 0 0 0]
[0 0 1 0 0 0 0]
[ 0  0 -1  0  0  0  0]
[ 0 -1  0  0  0  1  1]
[1 0 0 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[ 0 -1  0  0  0  0  0]
[ 0  0  0  0 -1  0  0]
[ 0  0  0  0 -1  0  0]
[0 0 0 0 0 1 0]
[ 0  0 -1  0  0  0  0]
[ 0  1  0  0  0  1 -1]
[0 0 1 0 0 0 0]
[0 0 0 0 0 1 0]
[ 0  1  0  0  0  1 -1]
[ 0  1 -1  0  0  1  0]
[ 0  1 -1  0  0  1  0]
[0 1 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 1 0]
[ 0  0 -1  0  1  1  0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[ 0  1  0  0  0  1 -1]
[ 0  0  0  0  0  0 -1]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[ 0  0  0  0  0  0 -1]
[0 0 0 1 0 0 0]
[0 1 0 0 0 0 0]
[ 1 -1  1  0  0  0  0]
[ 0 -1  0  0  0  0  0]
[1 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[ 0  1  1  0  0 -1  0]
[ 0  1  0  0 -1 -1  0]
[-1  0  0  0  0  0  0]
[0 0 1 0 0 0 0]
[0 0 0 0 0 1 0]
[0 0 0 1 0 1 0]
[0 0 1 0 0 0 0]
[ 1  0  0  0 -1  0 -1]
[0 1 0 0 0 0 0]
[-1  0  0 -1  0  0  0]
[0 0 1 0 0 0 1]
[ 0  

array([  6,   0,   9,   0,   1,   0, -57])

In [54]:
b

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0, -1,  0]])